# Imports

In [10]:
import numpy as np
import pandas as pd
import random as rand

from keras.utils import to_categorical
from keras.layers import Dense, Activation, Conv2D,Conv2DTranspose, Dropout, Reshape, MaxPooling2D, Flatten
from keras.models import Sequential

from sklearn.model_selection import train_test_split

# Data importing and pre-processing

In [2]:
df = pd.read_csv('dataset.csv', header=None)


df = df.values.reshape(60, 1, 64, 64)

labels = np.zeros(60)

x_real_train, x_real_test = train_test_split(df, test_size=0.2) #12 test values
y_real_train, y_real_test = train_test_split(labels, test_size=0.2)

# Generator Model

Create points in latent space to be fed into generator

In [3]:
def generate_generator_input(no_of_samples):
    input = np.random.randn(100*no_of_samples)

    input = input.reshape(60,100)

    return input

In [4]:
def make_generator():
    model = Sequential()
    
    model.add(Dense(60*8*8, input_dim=100))
    model.add(Activation("relu"))
    model.add(Reshape((8,8,60)))
    
    model.add(Conv2DTranspose(1, (1,1), strides=(2,2), padding='same', input_shape=(8,8,60)))
    model.add(Activation("relu"))
  
    
    model.add(Conv2DTranspose(1, (1,1), strides=(2,2), padding='same', input_shape=(16,16,60)))
    model.add(Activation("relu"))

    model.add(Conv2DTranspose(1, (1,1), strides=(2,2), padding='same', input_shape=(32,32,60)))
    model.add(Activation("relu"))
              
    model.summary()
    
    return model
    
    

# Discriminator Model

In [11]:

def make_discriminator():
    
    # 1st set of layers
    model = Sequential()
    model.add(Conv2D(128, kernel_size=5, padding="same", input_shape=(1,64,64)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
    
    
    
    # output layer
    model.add(Dense(1)) # Binary classification (2 outputs), so only 1 dense layer needed
    model.add(Activation("sigmoid"))

    model.add(Flatten())
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    
    return model
    

# Training

First, create the models from the functions

In [12]:
gen_model = make_generator()
disc_model = make_discriminator()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 3840)              387840    
                                                                 
 activation_6 (Activation)   (None, 3840)              0         
                                                                 
 reshape_1 (Reshape)         (None, 8, 8, 60)          0         
                                                                 
 conv2d_transpose_3 (Conv2D  (None, 16, 16, 1)         61        
 Transpose)                                                      
                                                                 
 activation_7 (Activation)   (None, 16, 16, 1)         0         
                                                                 
 conv2d_transpose_4 (Conv2D  (None, 32, 32, 1)         2         
 Transpose)                                           

Next, create fake training data, and put it together with the real data.

In [13]:
gen_input = generate_generator_input(60)

print(gen_input.shape)

x_fake = gen_model.predict(gen_input)
y_fake = np.ones((60,), dtype=int)

x_fake = x_fake.reshape(60, 1, 64, 64)
    
x_fake_train, x_fake_test = train_test_split(x_fake, test_size=0.2)
y_fake_train, y_fake_test = train_test_split(y_fake, test_size=0.2)

print("shapes:")
print("x_fake_train : " + str(x_fake_train.shape))
print("x_fake_test : " + str(x_fake_test.shape))
print("y_fake_train : " + str(y_fake_train.shape))
print("y_fake_test : " + str(y_fake_test.shape))

print("x_real_train : " + str(x_real_train.shape))
print("x_real_test : " + str(x_real_test.shape))
print("y_real_train : " + str(y_real_train.shape))
print("y_real_test : " + str(y_real_test.shape))

x_train = np.concatenate((x_fake_train, x_real_train))
y_train = np.concatenate((y_fake_train, y_real_train))
x_test = np.concatenate((x_fake_test, x_real_test))
y_test = np.concatenate((y_fake_test, y_real_test))

(60, 100)
2/2 [==============================] - 0s 19ms/step
shapes:
x_fake_train : (48, 1, 64, 64)
x_fake_test : (12, 1, 64, 64)
y_fake_train : (48,)
y_fake_test : (12,)
x_real_train : (48, 1, 64, 64)
x_real_test : (12, 1, 64, 64)
y_real_train : (48,)
y_real_test : (12,)


2023-08-04 07:43:18.705971: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 15698944 exceeds 10% of free system memory.


Now, train the discriminator

Disc training parameters:

In [14]:
N_EPOCHS = 12
BATCH_SIZE = 4
VERBOSE = 1


In [15]:
print(x_train.shape)
print(y_train.shape)

disc_model.compile(loss="binary_crossentropy")
disc_model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=N_EPOCHS, verbose=VERBOSE, validation_data=(x_test, y_test))

(96, 1, 64, 64)
(96,)
Epoch 1/12
24/24 [==============================] - 2s 15ms/step - loss: 0.6492 - val_loss: 0.3400
Epoch 2/12
24/24 [==============================] - 0s 7ms/step - loss: 0.3373 - val_loss: 0.3345
Epoch 3/12
24/24 [==============================] - 0s 6ms/step - loss: 0.3320 - val_loss: 0.3294
Epoch 4/12
24/24 [==============================] - 0s 6ms/step - loss: 0.3269 - val_loss: 0.3242
Epoch 5/12
24/24 [==============================] - 0s 7ms/step - loss: 0.3218 - val_loss: 0.3193
Epoch 6/12
24/24 [==============================] - 0s 7ms/step - loss: 0.3169 - val_loss: 0.3143
Epoch 7/12
24/24 [==============================] - 0s 7ms/step - loss: 0.3120 - val_loss: 0.3094
Epoch 8/12
24/24 [==============================] - 0s 7ms/step - loss: 0.3070 - val_loss: 0.3045
Epoch 9/12
24/24 [==============================] - 0s 6ms/step - loss: 0.3021 - val_loss: 0.2995
Epoch 10/12
24/24 [==============================] - 0s 7ms/step - loss: 0.2973 - val_loss: 0.2

Put models together into full GAN

In [ ]:
def make_GAN(gen,disc):
    disc.trainable = False

    model = Sequential()

    model.add(gen)
    model.add(disc)

    model.compile(loss='binary_crossentropy',optimizer='adam')

    model.summary()

    return model

Train GAN

In [ ]:
GAN_model = make_GAN(gen_model, disc_model)


# create some fake data as input

x_gan = generate_generator_input(60)
y_gan = np.ones((60))

GAN_model.fit(x_gan, y_gan)

Now that all training is done, generate an output value

In [ ]:
# first, create the input from noise
GAN_input = generate_generator_input(1)

GAN_output = GAN_model.predict(GAN_input)



Output result graphically